In [2]:
import torchvision.models as models
import torchvision.ops as ops
import torch
from torch import nn
from typing import OrderedDict

In [6]:
img = torch.zeros(1,3,128,1024)

def forward_first_layer(model, image):
        """Executing forward pass for the zeroth Retina Net layer"""
        x = model.conv1(image)
        x = model.bn1(x)
        x = model.relu(x)
        x = model.maxpool(x)
        return x

class Layer(nn.Sequential):
    def __init__(self,channels,layer_index):
        # [1, 512, 4, 32]
        super().__init__(
            nn.ReLU(),
            #[64, 128, 256, 512, 64, 64],
            #i= 512, o=64
            #i=64, o=64
            # next
            # i=64, o=64
            # i=64, o=64
            nn.Conv2d(in_channels=channels[layer_index-1], out_channels=channels[layer_index], kernel_size=1, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=channels[layer_index], out_channels=channels[layer_index], kernel_size=1, stride=2, padding=0),
            nn.ReLU(),
        )




In [8]:
#output_channels=[64, 128, 256, 512, 64, 64] # without fpn
out_channels = [256, 256, 256, 256, 512 , 256]

model = models.resnet34(pretrained=True)
layer5 = Layer(out_channels, 4)
layer6 = Layer(out_channels, 5)

fpn = ops.FeaturePyramidNetwork(out_channels, 256)
oout_features = []
features_dict = OrderedDict()

# Layer 0
x = forward_first_layer(model,img)
# [1, 64, 32, 256]

# Layer 1
features_dict['feat0'] = model.layer1(x)
# [1, 64, 32, 256] 

# Layer 2
features_dict['feat1'] = model.layer2(features_dict['feat0'])
# [1, 128, 16, 128]

# Layer 3
features_dict['feat2'] = model.layer3(features_dict['feat1'])
# [1, 256, 8, 64]

# Layer 4
features_dict['feat3'] = model.layer4(features_dict['feat2'])
# [1, 512, 4, 32]
# Dette vil vi ha? [1, 256, 4, 32]

print(features_dict['feat3'].shape)

# Layer 5
features_dict['feat4'] = layer5(features_dict['feat3'])
# [1, 64 , 2, 16]      

# Layer 6
features_dict['feat5'] = layer6(features_dict['feat4'])
# [1, 64 , 1, 8]
# Forward to FPN
out_features = fpn(features_dict)

print(out_features)




torch.Size([1, 512, 4, 32])


RuntimeError: Given groups=1, weight of size [512, 256, 1, 1], expected input[1, 512, 4, 32] to have 256 channels, but got 512 channels instead